In [ ]:
!pip install openai pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import pandas as pd
import requests
import json
from google.colab import files
# Load your existing data
data = pd.read_excel('50human100eachwithnumber.xlsx')

# Sample 10 random rows from the data
sampled_data = data.sample(n=100, random_state=42)  # Set random_state for reproducibility

# OpenAI API settings
#api_key =  "" #insert openai api key
api_url = "https://api.openai.com/v1/chat/completions"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

# Initialize a new DataFrame for the new data
new_data = pd.DataFrame(columns=['label', 'text', 'summary'])

# Function to call GPT-4 engine for summarization
def summarize_gpt(original_text):
    prompt = f"Tell me in general what the topic of the following article is just in 1 word(for example, Economy, Trade, Winter, Football, and so on, but do not say China, or Russia or country name): {original_text}"

    data = {
        'model': 'gpt-3.5-turbo',
        'messages': [
            {
                'role': 'system',
                'content': 'Tell me in general what the topic of the following article is just in 1 word(for example, Economy, Trade, Winter, Football, and so on, but do not say China, or Russia or country name)'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    }

    response = requests.post(api_url, headers=headers, json=data)
    response_json = response.json()

    if response.status_code == 200:
        return response_json['choices'][0]['message']['content'].strip()
    else:
        print("Error:", response.status_code, response_json)
        return None

# Function to call GPT-4 engine for generating a news article from a summary
def generate_gpt(summary):
    prompt = f"Write a news article of length between 200 and 250 words based on the following topic: {summary}"

    data = {
        'model': 'gpt-4',
        'messages': [
            {
                'role': 'system',
                'content': 'Write a news article of length between 200 and 250 words based on the following topic'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    }

    response = requests.post(api_url, headers=headers, json=data)
    response_json = response.json()

    if response.status_code == 200:
        return response_json['choices'][0]['message']['content'].strip()
    else:
        print("Error:", response.status_code, response_json)
        return None

# Function to process a batch of data
def process_batch(batch_data):
    processed_data = pd.DataFrame(columns=['label', 'text', 'summary'])
    for index, row in batch_data.iterrows():
        original_text = row['text']

        # Generate the summary
        summary = summarize_gpt(original_text)
        if summary:
            new_text = generate_gpt(summary)
            # Add the original text with 'True Author' label
            processed_data = processed_data.append({'label': row['label'], 'text': original_text, 'summary': summary}, ignore_index=True)

            if new_text:  # If GPT-4 generated text successfully
                # Add the GPT-4 text with '51' label
                processed_data = processed_data.append({'label': '51', 'text': new_text, 'summary': summary}, ignore_index=True)

    # Save the processed data to an Excel file
    filename = f'processed_data_sample.xlsx'
    processed_data.to_excel(filename, index=False)
    files.download(filename)

# Process the sampled data
process_batch(sampled_data)

<ipython-input-11-065419049797>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_data = processed_data.append({'label': row['label'], 'text': original_text, 'summary': summary}, ignore_index=True)
<ipython-input-11-065419049797>:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_data = processed_data.append({'label': '51', 'text': new_text, 'summary': summary}, ignore_index=True)
<ipython-input-11-065419049797>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_data = processed_data.append({'label': row['label'], 'text': original_text, 'summary': summary}, ignore_index=True)
<ipython-input-11-065419049797>:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

Error: 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}}


<ipython-input-11-065419049797>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_data = processed_data.append({'label': row['label'], 'text': original_text, 'summary': summary}, ignore_index=True)
<ipython-input-11-065419049797>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_data = processed_data.append({'label': row['label'], 'text': original_text, 'summary': summary}, ignore_index=True)
<ipython-input-11-065419049797>:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_data = processed_data.append({'label': '51', 'text': new_text, 'summary': summary}, ignore_index=True)
<ipython-input-11-065419049797>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>